In [1]:
import boto3

s3 = boto3.client('s3')

bucket_name = 'cyrille-dscrap-bucket'
file_name = 'processed/station_bikes/2023-09-10-2023-09-24/StationaryStations.csv'

try:
    s3.download_file(bucket_name, file_name, 'local-file-name')
except Exception as e:
    print(f"An error occurred: {e}")

An error occurred: The SSO session associated with this profile has expired or is otherwise invalid. To refresh this SSO session run aws sso login with the corresponding profile.
